In [1]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [43]:
repo_list = pd.DataFrame()
repo_list_query = f"""
SELECT repo_id, repo_name, repo_path from repo;
    """
repo_list = pd.read_sql_query(repo_list_query, con=engine)
print(repo_list)

      repo_id                   repo_name  \
0       26639                 s1p2016-lab   
1       27768    license-finder-buildpack   
2       27321         docs-service-backup   
3       27869                      vivace   
4       26717        pivotal-gemfire-ldap   
...       ...                         ...   
2706    26961  dataTx-gemfire-devOps-bash   
2707    26973                      fiscal   
2708    27274                        a11y   
2709    26130        philips-hue-resource   
2710    27399   pcf-bosh-terraforming-gcp   

                                  repo_path  
0     github.com/Pivotal-Field-Engineering/  
1       github.com/pivotal-cf-experimental/  
2                    github.com/pivotal-cf/  
3                        github.com/vmware/  
4     github.com/Pivotal-Field-Engineering/  
...                                     ...  
2706            github.com/pivotalservices/  
2707            github.com/pivotalservices/  
2708                 github.com/pivotal-cf/  

In [8]:
repo_id = 28141
repo_list = pd.DataFrame()
repo_list_query = f"""
SELECT repo_groups.rg_name from repo_groups, repo
    WHERE 
    repo.repo_id = {repo_id}
    AND repo.repo_group_id = repo_groups.repo_group_id;
    """
repo_list = pd.read_sql_query(repo_list_query, con=engine)
print(repo_list.rg_name[0])

concourse


In [65]:
i = 0
for index, repo in repo_list.iterrows():
    repo_id = repo['repo_id']
    print(repo_id)
    
    repo_list_commits_query = f"""
            SELECT DISTINCT COUNT(cmt_commit_hash) from commits
            WHERE 
                repo_id = {repo_id};
            """
    repo_list_commits = pd.read_sql_query(repo_list_commits_query, con=engine)
    print(repo_list_commits)
    
    i+=1
    if i == 10:
        break

26639
   count
0    168
27768
   count
0    745
27321
   count
0    141
27869
   count
0   2019
26717
   count
0     93
27545
   count
0    708
27726
   count
0     23
25856
   count
0    129
27774
   count
0    240
25793
   count
0    318


In [44]:
#from datetime import date
import datetime 

current = datetime.date.today()
today = "'" + str(current) + "'"
#print(today)

first_current = current.replace(day=1)
last_month = first_current - datetime.timedelta(days=1)
end_date = "'" + str(last_month) + "'"
#print(end_date)

start = last_month - datetime.timedelta(days=365) #### 6 MONTHS
start_date = "'" + str(start) + "'"

print(start_date, end_date)

'2019-03-01' '2020-02-29'


In [45]:
    repo_list_commits = pd.DataFrame()
    repo_list_commits_query = f"""
            SELECT repo.repo_id, repo.repo_name, repo.repo_path, repo.forked_from, COUNT(commits.cmt_id) from repo, commits
            WHERE 
                repo.repo_id = commits.repo_id
                AND commits.cmt_author_timestamp >= {start_date}
                AND commits.cmt_author_timestamp <= {end_date}
            GROUP BY repo.repo_id
            ORDER BY COUNT(commits.cmt_id);
            """
    repo_list_commits = pd.read_sql_query(repo_list_commits_query, con=engine)
    print(repo_list_commits)

      repo_id                 repo_name                    repo_path  \
0       25852                 test-temp  github.com/Spring-projects/   
1       27486      http-session-caching       github.com/pivotal-cf/   
2       25694           spring-flex-roo  github.com/Spring-projects/   
3       27615   docs-book-app-automator       github.com/pivotal-cf/   
4       26221          homebrew-tracker   github.com/pivotaltracker/   
...       ...                       ...                          ...   
1134    27050                    harbor          github.com/pivotal/   
1135    27497  azure-blobstore-resource       github.com/pivotal-cf/   
1136    25760               spring-boot  github.com/Spring-projects/   
1137    25663          spring-framework  github.com/Spring-projects/   
1138    27645                   cvelist       github.com/pivotal-cf/   

     forked_from   count  
0           None       1  
1           None       1  
2           None       1  
3           None       1  


In [46]:
    repo_list_commits = pd.DataFrame()
    repo_list_commits_query = f"""
            SELECT count(distinct commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path from repo, commits
            WHERE 
                repo.repo_id = commits.repo_id
                AND commits.cmt_author_timestamp >= {start_date}
                AND commits.cmt_author_timestamp <= {end_date}
            GROUP BY repo.repo_id
            ORDER BY count(distinct commits.cmt_commit_hash);
            """
    repo_list_commits = pd.read_sql_query(repo_list_commits_query, con=engine)
    repo_list_commits
    #repo_list_commits.loc[repo_list_commits.repo_id == 28031]
    #df.loc[df.a == 3
    #.loc["California","2013"]

,count,repo_id,repo_name,repo_path
0,1,27486,http-session-caching,github.com/pivotal-cf/
1,1,26977,pks-windows-aquamarker,github.com/pivotalservices/
2,1,26497,edu-store-redirect,github.com/platform-acceleration-lab/
3,1,25694,spring-flex-roo,github.com/Spring-projects/
4,1,28131,mockups,github.com/concourse/
...,...,...,...,...
1134,2577,28051,concourse,github.com/concourse/
1135,2645,27050,harbor,github.com/pivotal/
1136,4057,28118,charts,github.com/concourse/
1137,4895,25760,spring-boot,github.com/Spring-projects/


In [47]:
top = repo_list_commits.loc[repo_list_commits['count'] > 2400]

In [19]:
len(top.repo_id)

6

In [20]:
top

,count,repo_id,repo_name,repo_path
1134,2457,25663,spring-framework,github.com/Spring-projects/
1135,2576,28051,concourse,github.com/concourse/
1136,2645,27050,harbor,github.com/pivotal/
1137,4057,28118,charts,github.com/concourse/
1138,4895,25760,spring-boot,github.com/Spring-projects/
1139,11509,27645,cvelist,github.com/pivotal-cf/


In [21]:
    for index, repo in top.iterrows():
        print(repo['repo_id'], repo['repo_name'])

25663 spring-framework
28051 concourse
27050 harbor
28118 charts
25760 spring-boot
27645 cvelist


In [22]:
    commit_exp = pd.DataFrame()
    commit_exp_query = f"""
            SELECT DISTINCT cmt_commit_hash, commits.cmt_author_timestamp from commits
            WHERE 
                commits.repo_id = 26983
                AND commits.cmt_author_timestamp >= '2019-07-01'
                AND commits.cmt_author_timestamp <= '2019-08-01'
            ORDER BY commits.cmt_author_timestamp;
            """
    commit_exp = pd.read_sql_query(commit_exp_query, con=engine)
    print(commit_exp)

                             cmt_commit_hash      cmt_author_timestamp
0   7873a33622056c7f88d6c84dd505cd7667eca1de 2019-07-11 14:58:46+00:00
1   6f2121abd49dfdbe443450a6c707b8d45fcf8efc 2019-07-11 15:10:08+00:00
2   3308c09aba50edf328b84c9efd10077538bcead0 2019-07-11 15:20:07+00:00
3   9567149ccf4f007c207d55fe1790ae0d17007a39 2019-07-11 15:34:19+00:00
4   8250e2909e7f1ce19bc4ed8296a10a1174ab5ec3 2019-07-11 15:36:12+00:00
..                                       ...                       ...
92  0fc6028f9b965c3c857eb31a9a2ef56812dd9795 2019-07-27 15:15:11+00:00
93  6d99a22e25106e91bca4f96fd7bc9a79ba6c6ce3 2019-07-29 15:56:15+00:00
94  f5cce9ac4a968a0e440c36d410a621412dddf15a 2019-07-29 18:43:12+00:00
95  2e31d5d424f90517e01fe17e3895ac409a8d752b 2019-07-30 09:28:22+00:00
96  3c6d4b6a8b12b33614871c50108a11682d3177fe 2019-07-30 10:43:49+00:00

[97 rows x 2 columns]


In [23]:
    commit_exp = pd.DataFrame()
    commit_exp_query = f"""
            SELECT updated_at, repo_id from repo
            ORDER BY updated_at
            """
    commit_exp = pd.read_sql_query(commit_exp_query, con=engine)
    print(commit_exp)

     updated_at  repo_id
0          None    26459
1          None    26949
2          None    25690
3          None    27566
4          None    26748
...         ...      ...
2706       None    26961
2707       None    26973
2708       None    27274
2709       None    26130
2710       None    27399

[2711 rows x 2 columns]


In [66]:
    all_commits = pd.DataFrame()
    all_commits_query = f"""
            SELECT DISTINCT(commits.cmt_commit_hash), repo.repo_id, repo.repo_name, repo.repo_path, 
                commits.cmt_author_email, commits.cmt_author_timestamp from repo, commits
            WHERE 
                repo.repo_id = commits.repo_id
            GROUP BY repo.repo_id, commits.cmt_commit_hash, commits.cmt_author_email, commits.cmt_author_timestamp
            ORDER BY repo.repo_id;
            """
    all_commits = pd.read_sql_query(all_commits_query, con=engine)
    all_commits

,cmt_commit_hash,repo_id,repo_name,repo_path,cmt_author_email,cmt_author_timestamp
0,0097eea9b9cfa0605916fac225513c14a808c910,25430,rmq-0mq,github.com/rabbitmq/,sustrik@250bpm.com,2010-09-09 08:49:09+00:00
1,04b6ed16843b336ce6ab18a693c94bbaf669dcee,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-08-13 11:35:42+00:00
2,0a43d1cb58bab9162f50bd4c75bed5bd17bd51ae,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-11-13 18:14:44+00:00
3,0e937a3370247b8f65e7a3eed0f66aea7f255453,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-09-15 11:26:31+00:00
4,1436532e5e2e7381e184b52356dfb3dc3740b636,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-09-10 16:56:12+00:00
...,...,...,...,...,...,...
1235945,e67bcfb3e20425f8c4ce9698122aa107155f5c4a,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-14 00:41:15+00:00
1235946,eeae26add4a9b42101a56ad246342ecb94ee42b5,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-22 14:53:45+00:00
1235947,ef8c628ff1f15455c6f46aacfe3af9f43f499b85,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-14 02:29:16+00:00
1235948,f50afe58b89f7f252a0541a78593bda9e93b6537,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-09 21:55:11+00:00


In [68]:
#all_commits.loc[all_commits['repo_id']]
all_commits['cmt_author_timestamp'].min()
all_commits['cmt_author_timestamp'].max()
#all_commits = all_commits.reset_index().groupby([all_commits.cmt_author_timestamp]).max().set_index('index')
all_commits.loc[all_commits.groupby('repo_id').cmt_author_timestamp.idxmax()].sort_values('cmt_author_timestamp')

,cmt_commit_hash,repo_id,repo_name,repo_path,cmt_author_email,cmt_author_timestamp
39241,26e84dc10d4bf442660bb0212b3d428640c1f5b7,25455,rabbithub,github.com/rabbitmq/,tonygarnockjones@gmail.com,2010-09-05 22:15:13+00:00
1036433,c009a0a48a66d6fb0c6ccd3b51b20200fdb58655,27839,pyvco,github.com/vmware/,yann.hodique@gmail.com,2010-11-07 15:55:02+00:00
39424,ec16a25cc6175f4cdf10421f4ac488090d6a0696,25456,rabbitmq-ha,github.com/rabbitmq/,matthew@rabbitmq.com,2010-12-16 14:38:01+00:00
160261,b99de99785ce2bd1e8b88839d3e9a62e34dff659,25689,cloudfoundry-sandbox,github.com/Spring-projects/,peter@cacoethes.co.uk,2011-06-07 15:13:23+00:00
163602,cf988c19af52baf5c87b74805a6080bbcd70ccea,25702,protocol-fallback-demo,github.com/Spring-projects/,ozhurakousky@pivotal.io,2011-08-04 13:13:17+00:00
...,...,...,...,...,...,...
553815,ab2de82048fa54218a727d0b7b07d056665084f6,26392,aws-pcf-quickstart,github.com/cf-platform-eng/,ramonmakkelie@gmail.com,2020-03-25 16:11:49+00:00
1168232,b9b5d173b552d4516275ad92d8f55004fede7133,28037,octant,github.com/vmware-tanzu/,wayne@riotousliving.com,2020-03-25 16:13:47+00:00
767525,a14cf3dce6fc6f4312b356e26423016505356b12,27061,paving,github.com/pivotal/,jarchie@pivotal.io,2020-03-25 16:19:23+00:00
741386,f30932095c70a32c0f2c1df9d7af592ce537d2e2,27043,kpack,github.com/pivotal/,mmcnew@pivotal.io,2020-03-25 16:24:19+00:00


In [69]:
all_commits

,cmt_commit_hash,repo_id,repo_name,repo_path,cmt_author_email,cmt_author_timestamp
0,0097eea9b9cfa0605916fac225513c14a808c910,25430,rmq-0mq,github.com/rabbitmq/,sustrik@250bpm.com,2010-09-09 08:49:09+00:00
1,04b6ed16843b336ce6ab18a693c94bbaf669dcee,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-08-13 11:35:42+00:00
2,0a43d1cb58bab9162f50bd4c75bed5bd17bd51ae,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-11-13 18:14:44+00:00
3,0e937a3370247b8f65e7a3eed0f66aea7f255453,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-09-15 11:26:31+00:00
4,1436532e5e2e7381e184b52356dfb3dc3740b636,25430,rmq-0mq,github.com/rabbitmq/,mikeb@squaremobius.net,2010-09-10 16:56:12+00:00
...,...,...,...,...,...,...
1235945,e67bcfb3e20425f8c4ce9698122aa107155f5c4a,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-14 00:41:15+00:00
1235946,eeae26add4a9b42101a56ad246342ecb94ee42b5,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-22 14:53:45+00:00
1235947,ef8c628ff1f15455c6f46aacfe3af9f43f499b85,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-14 02:29:16+00:00
1235948,f50afe58b89f7f252a0541a78593bda9e93b6537,28141,boarding-pass,github.com/concourse/,suraci.alex@gmail.com,2020-01-09 21:55:11+00:00


In [88]:
all_commits.loc[all_commits['repo_id'] == 25430].cmt_author_email.value_counts()

mikeb@squaremobius.net      49
sustrik@250bpm.com          17
michael@clojurewerkz.org     2
luke@bakken.io               1
Name: cmt_author_email, dtype: int64

In [79]:
repo_list_commits.loc[repo_list_commits['count'] > 2400]

,count
